In this notebook, we construct feature space for all encounters. The feature type will include demographics information (age-continuous, sex-binary, race-binary, whether black), comorbidities (pre-selection, binary) and in-observation window features, including medications (pre-selection, binary), procedures (pre-selection, binary) and lab test results (pre-selection, continous values) as well as baseline SCr level (continuous).  

For comorbidities and medications, we will add upper level ontology to enhance the features.

1. Literature Reviews for AKI-related Comorbidities: diabetes, HIV/AIDS, CKD (stages 1-5), hypertension, chronic liver diseases, heart failure, gastrointestinal diseases. 
2. Literature Reviews for AKI-related Medications: Written in the nephrotoxical_drug_data in the common_var.py
3. Literature Reviews for AKI-related Procedures: cardiac surgery, abdominal surgery, orthopaedic surgery, anesthesia, mechanical ventilation, contrast-enhanced CT.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import sys
import os
sys.path.append(os.path.abspath("/home/lideyi/AKI_GNN/notebooks/utils"))
from common_var import pat_id_cols
from tqdm import tqdm
# enable progress_apply
tqdm.pandas()

# Read Patient ID DataFrame

In [2]:
# read patient id dataframe
onset_df = pd.read_csv('/blue/yonghui.wu/lideyi/AKI_GNN/raw_data/onset_df_cleaned.csv')

# format columns
onset_df[pat_id_cols + ['SEX', 'RACE']] = onset_df[pat_id_cols + ['SEX', 'RACE']].astype(str)
date_cols = ['ADMIT_DATE', 'DISCHARGE_DATE', 'OBSERVATION_WINDOW_START', 'PREDICTION_POINT']
for col in date_cols:
    onset_df[col] = pd.to_datetime(onset_df[col]).dt.date

In [3]:
# since we already have demographic information in the onset_df
# we can directly translate them into features
# Convert SEX column to binary
onset_df['SEX'] = onset_df['SEX'].apply(lambda x: 1 if x == 'M' else (0 if x == 'F' else np.random.randint(0, 2)))
# For RACE, label Black as 1 and Others as 0
onset_df['RACE'] = onset_df['RACE'].apply(lambda x: 1 if x == 'Black' else 0)

# Extract Medications

Reference: https://www.aafp.org/pubs/afp/issues/2008/0915/p743.html. We organize a dictionar, with key being a drug name, values being a dictionary, containing drug class and mechanisms and RXCUI (SDC).

Centers with bad medications (high missing rates): MCW

In [4]:
from extract_RXCUI import get_rxcui_list
from common_var import nephrotoxical_drug_data

In [5]:
# for each drug in the nephrotoxical_drug_data, we will extract the RXCUI code
drug_with_rxcui = {}
for drug_name, drug_property in nephrotoxical_drug_data.items():
    rxcui_list = get_rxcui_list(drug_name)
    drug_property['RXCUI'] = rxcui_list
    drug_with_rxcui[drug_name] = drug_property

In [6]:

# here we do not create separate functions for medication data
# maybe need to add more centers here
KUMC_use_cols = ['PATID', 'MEDADMIN_START_DATE"+PD.DATE_SHIFT"', 
            'MEDADMIN_STOP_DATE"+PD.DATE_SHIFT"', 'MEDADMIN_TYPE', 'MEDADMIN_CODE']
KUMC_med = pd.read_csv('/blue/yonghui.wu/hoyinchan/Data/data2022raw/KUMC_ORCALE/raw/AKI_AMED.csv', usecols=KUMC_use_cols)

In [7]:
# processing the medicaion data an concat them
med_cols_names = ['PATID', 'MEDADMIN_START_DATE', 'MEDADMIN_STOP_DATE', 'MEDADMIN_TYPE', 'MEDADMIN_CODE']
KUMC_med.columns = med_cols_names
# add center name column
KUMC_med['CENTER_NAME'] = 'KUMC'
# maybe add more centers here in the future so just keep it
med_df = pd.concat([KUMC_med], axis=0)
# format string columns
med_df[['PATID', 'MEDADMIN_TYPE', 'MEDADMIN_CODE']] = med_df[['PATID', 'MEDADMIN_TYPE', 'MEDADMIN_CODE']].astype(str)
# before we format time columns, we need to remove the rows that we do not care (not in drug_with_rxcui)
# Create a reverse mapping from RXCUI to drug name
rxcui_to_drug = {rxcui: drug for drug, properties in drug_with_rxcui.items() for rxcui in properties['RXCUI']}
# Translate MEDADMIN_CODE to drug name
med_df['DRUG_NAME'] = med_df['MEDADMIN_CODE'].map(rxcui_to_drug)
# Drop rows that cannot be translated
med_df = med_df.dropna(subset=['DRUG_NAME'])
med_df.drop(columns=['MEDADMIN_CODE'], inplace=True)

In [8]:
# now we can format the time columns
med_df['MEDADMIN_START_DATE'] = pd.to_datetime(med_df['MEDADMIN_START_DATE']).dt.date
med_df['MEDADMIN_STOP_DATE'] = pd.to_datetime(med_df['MEDADMIN_STOP_DATE']).dt.date

/scratch/local/51878489/ipykernel_1895792/1257268778.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  med_df['MEDADMIN_START_DATE'] = pd.to_datetime(med_df['MEDADMIN_START_DATE']).dt.date
/scratch/local/51878489/ipykernel_1895792/1257268778.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  med_df['MEDADMIN_STOP_DATE'] = pd.to_datetime(med_df['MEDADMIN_STOP_DATE']).dt.date


In [9]:
# merge on PATID and filter out the rows that are not in the observation windows
onset_med_df = onset_df.merge(med_df, on=['CENTER_NAME', 'PATID'], how='left')
onset_med_df = onset_med_df[(onset_med_df['MEDADMIN_START_DATE'] >= onset_med_df['OBSERVATION_WINDOW_START']) & (onset_med_df['MEDADMIN_START_DATE'] <= onset_med_df['PREDICTION_POINT']) | \
                      (onset_med_df['MEDADMIN_STOP_DATE'] >= onset_med_df['OBSERVATION_WINDOW_START']) & (onset_med_df['MEDADMIN_STOP_DATE'] <= onset_med_df['PREDICTION_POINT'])]  
# now we can create the medication feature, that is we need to pivot the table and turn in to binary
med_feature = onset_med_df[pat_id_cols + ['DRUG_NAME']].drop_duplicates()
# Pivot the med_feature dataframe
med_feature_pivot = med_feature.pivot_table(index=pat_id_cols, columns='DRUG_NAME', aggfunc='size', fill_value=0)
# Convert the pivot table to binary (1 if the patient used the drug, 0 otherwise)
med_feature_pivot = (med_feature_pivot > 0).astype(int)
# Reset the index to make it a regular dataframe
med_feature_pivot.reset_index(inplace=True)

In [10]:
def merge_and_filter(onset_df: pd.DataFrame, feature_df: pd.DataFrame, threshold: float) -> pd.DataFrame:
    """
    Merge the onset_df with feature_df and filter out the cols that have less than threshold 1 values
    """
    # we keep the original onset_df number of columns and merge the medication feature
    onset_df_fea_num = len(onset_df.columns)
    # merge
    onset_df = onset_df.merge(feature_df, on = pat_id_cols, how='left')
    # for those do not have a redcord in the observation window, we will fill them with 0
    onset_df.fillna(0, inplace=True)
    
    # drop columns of medications with the rate of 1 less then 1%
    # Calculate the threshold for 1%
    threshold = len(onset_df) * threshold
    # Get the columns to keep based on the threshold
    columns_to_keep = onset_df.columns[:onset_df_fea_num].tolist() + \
                    [col for col in onset_df.columns[onset_df_fea_num:] if onset_df[col].sum() >= threshold]
    # Filter the dataframe to keep only the desired columns
    onset_df = onset_df[columns_to_keep]
    return onset_df


In [11]:
# merge med and filter out the columns that have less than 1% 1 values
onset_df = merge_and_filter(onset_df, med_feature_pivot, 0.01)

In [12]:
aug_med_feautures = False
# then we want to augment the medication features with their nephrotoxic mechanisms
# we will create a new column for each mechanism
# we allow one mechanism effects can be added up
if aug_med_feautures:
    for drug_name, drug_property in tqdm(drug_with_rxcui.items()):
        if drug_name in onset_df.columns:
            mechanisms = drug_property['mechanism']
            for mech in mechanisms:
                if mech not in onset_df.columns:
                    onset_df[mech] = 0
                onset_df[mech] += onset_df[drug_name]

# Extract Lab Test Results

In [13]:
# here is different from previous method, we need to merge on encounterid since the dataframe is too large
KUMC_lab_cols = ['PATID', 'ENCOUNTERID', 'LAB_LOINC', 'SPECIMEN_DATE"+PD.DATE_SHIFT"', 'RESULT_NUM']
KUMC_lab = pd.read_csv('/blue/yonghui.wu/hoyinchan/Data/data2022raw/KUMC_ORCALE/raw/AKI_LAB.csv', usecols = KUMC_lab_cols)

In [14]:
# process before merging, so that we will not be out of memory
KUMC_lab.columns = ['PATID', 'ONSETS_ENCOUNTERID', 'LAB_LOINC', 'SPECIMEN_DATE', 'RESULT_NUM']
KUMC_lab.dropna(subset=['LAB_LOINC', 'RESULT_NUM'], inplace=True)
KUMC_lab["CENTER_NAME"] = 'KUMC'
# format column types
KUMC_lab[['PATID', 'ONSETS_ENCOUNTERID', 'LAB_LOINC']] = KUMC_lab[['PATID', 'ONSETS_ENCOUNTERID', 'LAB_LOINC']].astype(str)
KUMC_lab = KUMC_lab[(KUMC_lab.CENTER_NAME.isin(onset_df.CENTER_NAME)) & (KUMC_lab.PATID.isin(onset_df.PATID)) & \
    (KUMC_lab.ONSETS_ENCOUNTERID.isin(onset_df.ONSETS_ENCOUNTERID))]
# format time columns
KUMC_lab.loc[:, "SPECIMEN_DATE"] = pd.to_datetime(KUMC_lab["SPECIMEN_DATE"], format='%d-%b-%y').dt.date
# requrie that all the lab tests should before 2020
KUMC_lab = KUMC_lab[KUMC_lab['SPECIMEN_DATE'] < pd.to_datetime('2020-01-01').date()]
KUMC_lab.reset_index(drop=True, inplace=True)

In [15]:
# start to merge
onset_lab_df = onset_df.merge(KUMC_lab, on=pat_id_cols, how='left')

In [16]:
# filtered by observation window
onset_lab_df = onset_lab_df[(onset_lab_df.SPECIMEN_DATE >= onset_lab_df.OBSERVATION_WINDOW_START) & (onset_lab_df.SPECIMEN_DATE <= onset_lab_df.PREDICTION_POINT)] 

In [17]:
# sort onset_lab_df rows by pat_id_cols, LAB_LOINC and SPECIMEN_DATE
onset_lab_df = onset_lab_df.sort_values(by=pat_id_cols + ['LAB_LOINC', 'SPECIMEN_DATE'])
onset_lab_df.reset_index(drop=True, inplace=True)

In [18]:
# Pivot the onset_lab_df dataframe
lab_feature_pivot = onset_lab_df.pivot_table(index=pat_id_cols, columns='LAB_LOINC', 
                                             values='RESULT_NUM', aggfunc='last', fill_value=np.nan)
# Reset the index to make pat_id_cols as columns
lab_feature_pivot.reset_index(inplace=True)

In [19]:
# record the original feature number before 
onset_df_fea_num = len(onset_df.columns)
# merge the lab test feature
onset_df = onset_df.merge(lab_feature_pivot, on = pat_id_cols, how='left')
# drop columns of lab tests with missing rates more than 30%
lab_missing_rates = onset_df.isnull().mean()
lab_columns_to_drop = lab_missing_rates[lab_missing_rates > 0.3].index
onset_df.drop(columns=lab_columns_to_drop, inplace=True)

In [20]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# use MICE to impute the missing values

# Create the imputer object
imputer = IterativeImputer(max_iter=1000, random_state=0)

# Fit the imputer on the onset_df
imputer.fit(onset_df.iloc[:, onset_df_fea_num:])

# Transform the onset_df to fill missing values
onset_df.iloc[:, onset_df_fea_num:] = imputer.transform(onset_df.iloc[:, onset_df_fea_num:])

# Extract Procedures

1. Cardiac surgery (33016-33999, 34001-37799)  
2. Abdominal surgery (49000-49084, 49180-49255, 49320-49329, 49400-49465, 49491-49659, 49900-49900, 49904-49999)  
3. General anesthesia (00100-00222, 00300-00352, 00400-00474, 00500-00580, 00600-00670, 00700-00797, 00800-00882,00902-00952, 01112-01173, 01200-01274, 01320-01444, 01462-01522, 01610-01680, 01710-01782, 01810-01860, 01916-01942, 01951-01953, 01958-01969, 01990-01999)
4. Contrast-enhanced CT (70841, 70460, 70487, 72126, 70491, 71260, 73201, 72129, 73701, 74177, 72132,
70543, 70553, 70336, 72156, 73222, 71552, 73220, 72157, 73722, 74183, 73720, 72158, 72197)
5. Mechanical ventilation (94002-94005)


In [21]:
CPT_codes = {
    "Cardiac surgery": [(33016, 33999), (34001, 37799)],
    
    "Abdominal_surgery": [
        (49000, 49084), (49180, 49255), (49320, 49329), 
        (49400, 49465), (49491, 49659), (49900, 49900), (49904, 49999)
    ],
    
    "General anesthesia": [
        (100, 222), (300, 352), (400, 474), (500, 580), 
        (600, 670), (700, 797), (800, 882), (902, 952), 
        (1112, 1173), (1200, 1274), (1320, 1444), (1462, 1522), 
        (1610, 1680), (1710, 1782), (1810, 1860), (1916, 1942), 
        (1951, 1953), (1958, 1969), (1990, 1999)
    ],
    
    "Contrast-enhanced CT":  [70841, 70460, 70487, 72126, 70491, 71260, 73201, 72129, 73701, 74177, 72132,
70543, 70553, 70336, 72156, 73222, 71552, 73220, 72157, 73722, 74183, 73720, 72158, 72197],
    
    "Mechanical_ventilation": [(94002, 94005)],
}   

In [22]:
KUMC_PX_cols = ['PATID', 'PX_DATE"+PD.DATE_SHIFT"', 'PX']
KUMC_PX_df = pd.read_csv('/blue/yonghui.wu/hoyinchan/Data/data2022raw/KUMC_ORCALE/raw/AKI_PX.csv', usecols = KUMC_PX_cols)

/scratch/local/51878489/ipykernel_1895792/895864485.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  KUMC_PX_df = pd.read_csv('/blue/yonghui.wu/hoyinchan/Data/data2022raw/KUMC_ORCALE/raw/AKI_PX.csv', usecols = KUMC_PX_cols)


In [23]:
# format column names and data types
KUMC_PX_df.columns = ['PATID', 'PX_DATE', 'PX']
KUMC_PX_df['CENTER_NAME'] = 'KUMC'
KUMC_PX_df[['PATID', 'PX']] = KUMC_PX_df[['PATID', 'PX']].astype(str)
KUMC_PX_df['PX_DATE'] = pd.to_datetime(KUMC_PX_df['PX_DATE'], format = '%d-%b-%y').dt.date

In [24]:
# check if the CPT code is in the specified range
def CPT_in_ranges(cpt_code, ranges):
    try:
        cpt_int = int(cpt_code)
        for r in ranges:
            if isinstance(r, tuple) and r[0] <= cpt_int <= r[1]:
                return True
        return False
    except ValueError:  # if the CPT code is not an integer
        return False

# check if the CPT code is in the specified values
def CPT_in_values(cpt_code, values):
    return cpt_code in map(str, values)


In [25]:
# filter by the CPT codes we are insterested in
filtered_PX_rows = []
for PX, codes in tqdm(CPT_codes.items()):
    if PX == "Contrast_enhanced_CT":
        filtered = KUMC_PX_df[KUMC_PX_df['PX'].apply(lambda x: CPT_in_values(x, codes))]
    else:  # for other procedures, check if the CPT code is in the specified range
        filtered = KUMC_PX_df[KUMC_PX_df['PX'].apply(lambda x: CPT_in_ranges(x, codes))]
    filtered_PX_rows.append(filtered)

# concat the filtered rows and drop duplicates
filtered_PX_df = pd.concat(filtered_PX_rows).drop_duplicates()

100%|██████████| 5/5 [01:31<00:00, 18.32s/it]


In [26]:
# merge the onset_df with the filtered_PX_df
onset_PX_df = onset_df.merge(filtered_PX_df, on=['CENTER_NAME', 'PATID'], how='left')

In [27]:
# filter by observation window
onset_PX_df = onset_PX_df[(onset_PX_df['PX_DATE'] >= onset_PX_df['OBSERVATION_WINDOW_START']) & (onset_PX_df['PX_DATE'] <= onset_PX_df['PREDICTION_POINT'])]
# pivot onset_PX_df to get the procedure feature, values as binary, index as pat_id_cols and columns as PX
PX_feature_pivot = onset_PX_df.pivot_table(index=pat_id_cols, columns='PX', aggfunc='size', fill_value=0)

In [28]:
# Convert the pivot table to binary (1 if the patient had the procedure, 0 otherwise)
PX_feature_pivot = (PX_feature_pivot > 0).astype(int)
# Reset the index to make it a regular dataframe
PX_feature_pivot.reset_index(inplace=True)

In [29]:
# merge PX and filter out the columns that have less than 1% 1 values
onset_df = merge_and_filter(onset_df, PX_feature_pivot, 0.01)

# Extract Comorbidities

1. Diabetes: {ICD9: 250, ICD10: E08-E13}.
2. HIV/AIDS: {ICD9: 042, V08, ICD10: B20-B24}.
3. CKD: {ICD9: 585, ICD10: N18}. 
4. Hypertensive diseases: {ICD9: 401-405, ICD10: I10-I16, I1A}. 
5. Chronic liver diseases: {ICD9: 571, ICD10: K70-K77}.
6. Heart failure: {ICD9: 428, ICD10: I50}. 

In [30]:
# since ICD9 codes will always be different than ICD10 codes, we can mix them here
DX_codes = {
    'Diabetes': ['250', 'E08', 'E09', 'E10', 'E11', 'E13'],
    'HIV': ['042', 'B20', 'B21', 'B22', 'B23', 'B24'],
    'CKD-1': ['585.1', 'N18.1'],
    'CKD-2': ['585.2', 'N18.2'],
    'CKD-3': ['585.3', 'N18.3'],
    'CKD-4': ['585.4', 'N18.4'],
    'CKD-5': ['585.5', 'N18.5'],
    'Hypertensive diseases': ['401', '402', '403', '404', '405', 'I10', 'I11', 'I12', 'I13', 'I15', 'I16', 'I1A'],
    'Chronic liver diseases': ['571', 'K70', 'K71', 'K72', 'K73', 'K74', 'K76', 'K77'],
    'Heart failure': ['428', 'I50'],
}

In [31]:
KUMC_DX_cols = ['PATID', 'DX', 'DX_DATE"+PD.DATE_SHIFT"']
KUMC_DX_df = pd.read_csv('/blue/yonghui.wu/hoyinchan/Data/data2022raw/KUMC_ORCALE/raw/AKI_DX.csv', usecols = KUMC_DX_cols)

In [32]:
# format the dataframe
KUMC_DX_df.columns = ['PATID', 'DX_DATE', 'DX']
KUMC_DX_df['CENTER_NAME'] = 'KUMC'
KUMC_DX_df[['PATID', 'DX']] = KUMC_DX_df[['PATID', 'DX']].astype(str)
KUMC_DX_df['DX_DATE'] = pd.to_datetime(KUMC_DX_df['DX_DATE'], format = '%d-%b-%y').dt.date
KUMC_DX_df.dropna(inplace = True)


In [33]:
def map_to_disease(DX):
    # DX_codes here is a global variable
    for disease, codes in DX_codes.items():
        if any(DX.startswith(code) for code in codes):
            return disease
    return None

In [34]:
# map the DX to disease
KUMC_DX_df.loc[:, 'DX_NAME'] = KUMC_DX_df['DX'].progress_apply(map_to_disease)
# delete the rows that cannot be mapped to any disease
filtered_DX_df = KUMC_DX_df.dropna(subset=['DX_NAME']).reset_index(drop=True)
filtered_DX_df.drop('DX', axis=1, inplace=True)


100%|██████████| 34471063/34471063 [02:54<00:00, 197938.14it/s]


In [35]:
# merge the onset_df with the filtered_DX_df
onset_DX_df = onset_df.merge(filtered_DX_df, on=['CENTER_NAME', 'PATID'], how='left')
# filter by admission date
onset_DX_df = onset_DX_df[onset_DX_df['DX_DATE'] < onset_DX_df['ADMIT_DATE']]

In [36]:
# pivot onset_DX_df to get the diagnosis feature, values as binary, index as pat_id_cols and columns as DX
DX_feature_pivot = onset_DX_df.pivot_table(index=pat_id_cols, columns='DX_NAME', aggfunc='size', fill_value=0)
# Convert the pivot table to binary (1 if the patient had the procedure, 0 otherwise)
DX_feature_pivot = (DX_feature_pivot > 0).astype(int)
# Reset the index to make it a regular dataframe
DX_feature_pivot.reset_index(inplace=True)

In [37]:
# merge med and filter out the columns that have less than 1% 1 values
onset_df_full = merge_and_filter(onset_df, DX_feature_pivot, 0.01)

In [38]:
# check onset_df_full contains NaN values
assert onset_df_full.isnull().sum().sum() == 0

# Output Data

In [39]:
# check the time range of the onset_df_full
print(onset_df_full.DISCHARGE_DATE.min(), onset_df_full.DISCHARGE_DATE.max())

2009-01-14 2019-12-31


In [40]:
# create 2 new columns to indicate test set and val set, since we are using temporal split, ADMIT_DATE
# we want the set the val and test sets to be the last 20% of the data, respectively
onset_df_full['TRAIN_SET'] = 0
onset_df_full['VAL_SET'] = 0
onset_df_full['TEST_SET'] = 0

onset_df_full.loc[onset_df_full.ADMIT_DATE < pd.to_datetime('2016-01-01').date(), 'TRAIN_SET'] = 1
onset_df_full.loc[(onset_df_full.ADMIT_DATE >= pd.to_datetime('2016-01-01').date()) & \
                 (onset_df_full.ADMIT_DATE < pd.to_datetime('2018-01-01').date()), 'VAL_SET'] = 1
onset_df_full.loc[onset_df_full.ADMIT_DATE >= pd.to_datetime('2018-01-01').date(), 'TEST_SET'] = 1

# check that the ones in TRAIN_SET, VAL_SET, TEST_SET cover the whole dataset
assert (onset_df_full[['TRAIN_SET', 'VAL_SET', 'TEST_SET']].sum(axis=1) == 1).all()



In [41]:
print("Time range:")
print('Train: ', onset_df_full[onset_df_full.TRAIN_SET == 1].ADMIT_DATE.min(), onset_df_full[onset_df_full.TRAIN_SET == 1].ADMIT_DATE.max())
print('Val: ', onset_df_full[onset_df_full.VAL_SET == 1].ADMIT_DATE.min(), onset_df_full[onset_df_full.VAL_SET == 1].ADMIT_DATE.max())
print('Test: ', onset_df_full[onset_df_full.TEST_SET == 1].ADMIT_DATE.min(), onset_df_full[onset_df_full.TEST_SET == 1].ADMIT_DATE.max())
print("Proportions:")
print('Train: ', len(onset_df_full[onset_df_full.TRAIN_SET == 1]) /len(onset_df_full))
print('Val: ', len(onset_df_full[onset_df_full.VAL_SET == 1]) /len(onset_df_full))
print('Test: ', len(onset_df_full[onset_df_full.TEST_SET == 1]) /len(onset_df_full))

Time range:
Train:  2009-01-10 2015-12-31
Val:  2016-01-01 2017-12-31
Test:  2018-01-01 2019-12-28
Proportions:
Train:  0.5496950823926301
Val:  0.21911013600386897
Test:  0.23119478160350096


In [42]:
# drop the columns that are not needed before normalization
# we also exclude BASELINE_SCR becasue AKI was labeled by SCR as well as some patients might not have a baseline SCR
cols_to_drop = ['CENTER_NAME', 'PATID', 'ONSETS_ENCOUNTERID', 'BASELINE_SCR', 'ADMIT_DATE', 'DISCHARGE_DATE', 
                'OBSERVATION_WINDOW_START', 'PREDICTION_POINT']
onset_df_full.drop(columns=cols_to_drop, inplace=True)

In [43]:
# # select a portion of the onset_df_full to run pilot experiments
# # we use random sampling here, with a percentatge of 50%
np.random.seed(88)
onset_df_pilot = onset_df_full.sample(frac=0.5).copy(deep = True)

In [44]:
def min_max_normalize_by_set(df: pd.DataFrame, features: list) -> pd.DataFrame:
    train_norm = df[df['TRAIN_SET'] == 1].copy(deep = True)
    val_norm = df[df['VAL_SET'] == 1].copy(deep = True)
    test_norm = df[df['TEST_SET'] == 1].copy(deep = True)
    
    train_norm[features] = min_max_normalize(train_norm[features])
    val_norm[features] = min_max_normalize(val_norm[features])
    test_norm[features] = min_max_normalize(test_norm[features])
    
    norm_df = pd.concat([train_norm, val_norm, test_norm]).sort_index()
    return norm_df

In [45]:
def min_max_normalize(df_subset: pd.DataFrame) -> pd.DataFrame:
    # Calculate min and max for each column
    min_vals = df_subset.min()
    max_vals = df_subset.max()
    range_vals = max_vals - min_vals
    
    # Avoid division by zero by replacing ranges of 0 with 1
    range_vals = range_vals.replace(0, 1)
    
    # Apply Min-Max normalization
    return (df_subset - min_vals) / range_vals

In [46]:
feature_columns = [col for col in onset_df_full.columns if col not in ['AKI_TARGET', 'TRAIN_SET', 'VAL_SET', 'TEST_SET']]
norm_onset_df_full = min_max_normalize_by_set(onset_df_full, feature_columns)
norm_onset_df_pilot = min_max_normalize_by_set(onset_df_pilot, feature_columns)

In [47]:
norm_onset_df_full

,AGE,SEX,RACE,AKI_ONSET_LABEL,acetaminophen,acyclovir,alprazolam,amitriptyline,amoxicillin,aspirin,atorvastatin,ciprofloxacin,clonazepam,clopidogrel,diphenhydramine,fluoxetine,ibuprofen,ketamine,lansoprazole,levofloxacin,pantoprazole,sulfamethoxazole,tacrolimus,vancomycin,17861-6,1963-8,2075-0,2160-0,2345-7,26464-8,2823-3,2951-2,3094-0,32623-1,33037-3,4544-3,48642-3,48643-1,718-7,777-3,785-6,786-4,787-2,788-0,789-8,01996,1996,36415,36430,36556,36569,36600,36620,94002,94003,CKD-2,CKD-3,Chronic liver diseases,Diabetes,Heart failure,Hypertensive diseases,TRAIN_SET,VAL_SET,TEST_SET
0,0.704225,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.375000,0.348837,0.453125,0.179420,0.117078,0.014281,0.327273,0.561644,0.031646,0.272727,0.342105,0.180077,0.521277,0.425926,0.215116,0.021858,0.592834,0.587245,0.514286,0.456359,0.147541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0
1,0.704225,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.446429,0.418605,0.375000,0.224274,0.108485,0.017527,0.327273,0.520548,0.063291,0.272727,0.315789,0.298851,0.521277,0.425926,0.331395,0.018215,0.625407,0.587245,0.560000,0.461347,0.229508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0
2,0.295775,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.344828,0.461538,0.535714,0.083333,0.097701,0.028857,0.346939,0.480769,0.043478,0.171429,0.209302,0.452471,0.716418,0.589744,0.462857,0.323416,0.563694,0.712963,0.509653,0.153584,0.376296,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0,0,1
3,0.295775,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.298851,0.487179,0.482143,0.155914,0.103448,0.017079,0.408163,0.423077,0.043478,0.180952,0.186047,0.435361,0.716418,0.589744,0.405714,0.311846,0.471338,0.500000,0.482625,0.180887,0.374815,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0,0,1
4,0.295775,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.275862,0.333333,0.589286,0.102151,0.159483,0.027091,0.489796,0.403846,0.024845,0.276190,0.162791,0.408745,0.716418,0.589744,0.394286,0.120110,0.538217,0.555556,0.540541,0.156997,0.328889,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169549,0.830986,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.448276,0.692308,0.464286,0.077957,0.170977,0.040636,0.306122,0.500000,0.074534,0.580952,0.116279,0.528517,0.716418,0.589744,0.508571,0.095317,0.461783,0.555556,0.447876,0.177474,0.469630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
169550,0.450704,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.436782,0.538462,0.482143,0.153226,0.120690,0.035041,0.367347,0.423077,0.099379,0.447619,0.139535,0.389734,0.716418,0.589744,0.400000,0.104683,0.515924,0.712963,0.451737,0.078498,0.348148,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,0,1
169551,0.295775,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.413793,0.461538,0.625000,0.099462,0.125000,0.035041,0.346939,0.461538,0.012422,0.171429,0.069767,0.397338,0.716418,0.589744,0.382857,0.332231,0.442675,0.583333,0.411840,0.194539,0.370370,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
169552,0.309859,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.448276,0.564103,0.535714,0.126344,0.126437,0.027385,0.387755,0.519231,0.049689,0.257143,0.162791,0.425856,0.716418,0.589744,0.405714,0

In [48]:
# check that the ones in TRAIN_SET, VAL_SET, TEST_SET cover the whole dataset
assert (norm_onset_df_full[['TRAIN_SET', 'VAL_SET', 'TEST_SET']].sum(axis=1) == 1).all()
assert (norm_onset_df_pilot[['TRAIN_SET', 'VAL_SET', 'TEST_SET']].sum(axis=1) == 1).all()

# check that norm_onset_df_full and norm_onset_df_pilot do not have Nan values
assert norm_onset_df_full.isnull().sum().sum() == 0
assert norm_onset_df_pilot.isnull().sum().sum() == 0

In [49]:
norm_onset_df_pilot[norm_onset_df_pilot.TRAIN_SET == 1].describe()

,AGE,SEX,RACE,AKI_ONSET_LABEL,acetaminophen,acyclovir,alprazolam,amitriptyline,amoxicillin,aspirin,atorvastatin,ciprofloxacin,clonazepam,clopidogrel,diphenhydramine,fluoxetine,ibuprofen,ketamine,lansoprazole,levofloxacin,pantoprazole,sulfamethoxazole,tacrolimus,vancomycin,17861-6,1963-8,2075-0,2160-0,2345-7,26464-8,2823-3,2951-2,3094-0,32623-1,33037-3,4544-3,48642-3,48643-1,718-7,777-3,785-6,786-4,787-2,788-0,789-8,01996,1996,36415,36430,36556,36569,36600,36620,94002,94003,CKD-2,CKD-3,Chronic liver diseases,Diabetes,Heart failure,Hypertensive diseases,TRAIN_SET,VAL_SET,TEST_SET
count,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.000000,46491.0,46491.0,46491.0
mean,0.554902,0.499365,0.140221,0.145082,0.485234,0.052462,0.045923,0.024779,0.010389,0.235874,0.093480,0.019380,0.035319,0.036717,0.119937,0.020348,0.034286,0.003958,0.083371,0.129165,0.294337,0.025575,0.012583,0.006431,0.370877,0.433681,0.484095,0.202256,0.100941,0.031738,0.366736,0.441941,0.092849,0.328211,0.179980,0.429015,0.497374,0.415600,0.462077,0.133414,0.488855,0.550789,0.453349,0.290554,0.388073,0.021122,0.011615,0.305629,0.033985,0.013874,0.018326,0.010260,0.061044,0.011658,0.013207,0.008733,0.027726,0.077865,0.178099,0.089222,0.360478,1.0,0.0,0.0
std,0.235595,0.500005,0.347220,0.352187,0.499787,0.222959,0.209320,0.155453,0.101397,0.424548,0.291108,0.137858,0.184586,0.188068,0.324892,0.141189,0.181965,0.062787,0.276445,0.335386,0.455749,0.157865,0.111468,0.079938,0.053945,0.081253,0.074807,0.082364,0.045418,0.019616,0.082041,0.066002,0.060579,0.105187,0.058929,0.126692,0.065078,0.040370,0.126506,0.066383,0.086656,0.057245,0.083079,0.068054,0.118000,0.143794,0.107147,0.460678,0.181193,0.116968,0.134129,0.100772,0.239414,0.107343,0.114161,0.093042,0.164188,0.267961,0.382600,0.285066,0.480144,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0
25%,0.408451,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.339286,0.395349,0.450000,0.150794,0.073514,0.022134,0.320000,0.411765,0.055172,0.270000,0.131579,0.339583,0.510638,0.425926,0.372671,0.095034,0.448161,0.528280,0.410872,0.244389,0.302013,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0
50%,0.577465,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.375000,0.433395,0.483333,0.195767,0.092973,0.030842,0.368884,0.444051,0.082950,0.320000,0.184211,0.434975,0.5

In [50]:
# save the normalized dataframe. This is the final dataframe we will use for training
norm_onset_df_pilot.to_csv('/blue/yonghui.wu/lideyi/AKI_GNN/raw_data/norm_df_pilot.csv', index=False)
norm_onset_df_full.to_csv('/blue/yonghui.wu/lideyi/AKI_GNN/raw_data/norm_df_full.csv', index=False)